In [3]:
import torch
import onnx
import tensorrt as trt
from mmcv.tensorrt import (TRTWrapper, onnx2trt, save_trt_engine,
                                   is_tensorrt_plugin_loaded)

assert is_tensorrt_plugin_loaded(), 'Requires to complie TensorRT plugins in mmcv'

In [4]:
onnx_file = 'hrnet.onnx'
trt_file = 'hrnet.trt'

In [5]:
onnx_model = onnx.load(onnx_file)


In [4]:
# HRNet [1, 3, 256, 192]
opt_shape_dict = {
    'img': [[1, 3, 256, 192],
           [1, 3, 256, 192],
           [4, 3, 256, 192]]
}

In [5]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

max_workspace_size = 1 << 30
trt_engine = onnx2trt(
    onnx_model,
    opt_shape_dict,
    log_level=trt.Logger.INFO,
    max_workspace_size=max_workspace_size)

[TensorRT] INFO: Detected 1 inputs and 1 output network tensors.


In [6]:
save_trt_engine(trt_engine, trt_file)

In [35]:
trt_model = TRTWrapper(trt_file, ['input.1'], ['116'])
trt_model = trt_model.to(torch.device("cuda:1"))

In [36]:
for param in trt_model.parameters():
    print(param)

In [ ]:
import numpy as np
import pickle
from mmpose.core.evaluation.top_down_eval import keypoints_from_heatmaps 

In [ ]:
f = open("3d_img.p", "rb")
batch_data = pickle.load(f)
f.close()

In [ ]:
import time
t1 = time.time()
with torch.no_grad():
    trt_outputs = trt_model({'input.1': batch_data["input"][0][None, :]})
    output = trt_outputs['116']
    poses_3d = output.cpu().detach().numpy()
    
    if poses_3d.shape[-1] != 4:
        assert poses_3d.shape[-1] == 3
        dummy_score = np.ones(
            poses_3d.shape[:-1] + (1, ), dtype=poses_3d.dtype)
        poses_3d = np.concatenate((poses_3d, dummy_score), axis=-1)
    pose_results = []
    for pose_2d, pose_3d in zip(pose_sequences_2d, poses_3d):
        pose_result = pose_2d.copy()
        pose_result['keypoints_3d'] = pose_3d
        pose_results.append(pose_result)
        
    print(output_np)
print(f"Time: {time.time() - t1}")

In [ ]:
keypoints_2d = keypoints[0][0]

In [ ]:
import cv2
from matplotlib import pyplot as plt
img = cv2.imread("test.jpg")

In [ ]:
def show_keypoints(img, keypoints):
    for i in range(17):
        img = cv2.circle(img, (int(keypoints[i][0]), int(keypoints[i][1])), 3, (255, 0, 0), 1)
        
    return img

In [ ]:
img = show_keypoints(img, keypoints_2d)

In [ ]:
plt.imshow(img)

In [ ]:

device = trt_model.parameters()

In [ ]:
device

In [ ]:
for x in device:
    print(x)

#### Load

In [ ]:
from mmdet.apis import inference_detector
from mmdet2trt.apis import create_wrap_detector
from mmcv.tensorrt import (TRTWrapper, onnx2trt, save_trt_engine,
                                   is_tensorrt_plugin_loaded)

In [ ]:
trt_detector = create_wrap_detector("faster_rcnn.trt", "demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py", "cuda:0")


# result share same format as mmdetection
# 


In [ ]:
import cv2
frame = cv2.imread("test.jpg")
print("Read frame")
print(frame)
print(next(trt_detector.parameters()).device)

In [ ]:
result = inference_detector(trt_detector, frame)